In [169]:
# The code was removed by Watson Studio for sharing.

<h1>Coursera Capstone Project</h1>
<hr>
<h2>Exploring location data use</h2>

Import the required libraries

In [170]:
#!conda install --override-channels -c conda-forge/noarch geopy
#!conda install --override-channels -c conda-forge/noarch folium
#print('Folium installed')

In [171]:
!pip install geopy folium
!pip install lxml html5lib beautifulsoup4
!pip install geocoder geopandas
print('All installed')

All installed


In [172]:
!pip install geolocator

In [173]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import geopandas

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Libraries imported.')

Libraries imported.


<h1>Task 1</h1>
<h2>Scrap website and retrieve table with postal codes</h2>

In [174]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df = dfs[0][['Postal Code', 'Borough', 'Neighborhood']]

<h3>Clean up dataset</h3>

In [175]:
dftoronto = df[df['Borough']!= 'Not assigned']

#Use group by with join to remove duplicate postal codes
dftoronto = dftoronto.groupby('Postal Code').agg({'Borough': ', '.join, 
                             'Neighborhood': ', '.join}).reset_index()

#Take care of Not assigned Neighborhood
dftoronto = dftoronto[dftoronto['Neighborhood']!= 'Not assigned']
dftoronto.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [176]:
print('Shapre of dataframe: ', dftoronto.shape)

Shapre of dataframe:  (103, 3)


<h1>Task 2</h1>
<h2>Add geospatial data</h2>

In [177]:
#Try to get latlong with any of providers failing
#import geocoder
#initialize

#for index, row in dftoronto.iterrows():
#    # initialize your variable to None
#    lat_lng_coords = None
#    g = geopandas.tools.geocode('{}, Toronto, Ontario'.format(row['Postal Code']), provider='nominatim', user_agent="my-application")
#    if g is not None:
#        latitude = g.geometry[0].x
#        longitude = g.geometry[0].y
#
#        dftoronto['Latitude'] = latitude
#        dftoronto['Longitude'] = longitude


In [178]:
dfcodes = pd.read_csv('https://cocl.us/Geospatial_data')

In [179]:
dftoronto = pd.concat([dftoronto, dfcodes], axis = 1, join='inner')
dftoronto.head()

,Postal Code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [180]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'dftoronto.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'datascience-donotdelete-pr-lmuvdeywto5fgx',
 'asset_id': 'a8581b56-e2a8-4d8d-b48b-4d0d053327ad'}

<h1>Task 3</h1>
<h2>Clustering the neighbourhood</h2>

<h2>Vizualize the data</h2>

In [181]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dftoronto['Latitude'], dftoronto['Longitude'], dftoronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<h2>Explore neighborhood</h2>

In [182]:
# The code was removed by Watson Studio for sharing.

Foursquare credentails hidden.


In [183]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [184]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [185]:
# type your answer here

toronto_venues = getNearbyVenues(names=dftoronto['Neighborhood'],
                                   latitudes=dftoronto['Latitude'],
                                   longitudes=dftoronto['Longitude']
                                  )

In [186]:
print(toronto_venues.shape)
toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

(1335, 7)
There are 233 uniques categories.


<h2>Analyze Each Neighborhood</h2>

In [187]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [188]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#group by 
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

toronto_grouped.shape

(96, 233)

In [189]:

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Electronics Store
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Dance Studio,Gym,Skating Rink
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Sushi Restaurant,Pharmacy,Pizza Place
3,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Women's Store
4,"Bedford Park, Lawrence Manor East",Sushi Restaurant,Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant


<h2>Clustering</h2>

In [190]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [191]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dftoronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged.dropna()
toronto_merged.head()

,Postal Code,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353,5.0,Fast Food Restaurant,Women's Store,Dance Studio,Eastern European Restaurant,Drugstore
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497,6.0,Bar,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711,1.0,Electronics Store,Mexican Restaurant,Breakfast Spot,Medical Center,Moving Target
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Convenience Store,Mexican Restaurant,Women's Store
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,1.0,Lounge,Bakery,Bank,Athletics & Sports,Hakka Restaurant


<h2>Vizualize clusters</h2>

In [192]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [193]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters